In [1]:
import os

os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
from Models import run_F4
from Data.Data_Parser import fetch_all_dataset
from Solution.Map_Printer import osm_print_cluster_tours, COLOR_LIST
from Cluster_MILP import preprocess_clusters, calculate_arcs, fetch_arcs, cluster_MILP
from cluster import kmeans, kmedoids, kmeans_DBSCAN, som_cluster, p_median
from IPython.display import IFrame
import pickle
import json
import math
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

### Do cluster here

In [7]:
data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands = fetch_all_dataset("Paris")

  0%|          | 0/113 [00:00<?, ?it/s]

  0%|          | 0/12769 [00:00<?, ?it/s]

In [8]:
cluster_result = p_median(data, V_c, 9, runtime_limit=1800, print_out=False)
# _, cluster_result = kmeans(data) # new center
# cluster_result = kmedoids(data, 10)
# cluster_result = kmeans_DBSCAN(data) # new center

# df = pd.read_csv(".\\Data\\Paris.nodes", sep=' ')
# cluster_result = som_cluster(data, df, map_shape=(3, 3), num_iteration=100000)

In [9]:
result_dict, new_vertices, new_demands = cluster_MILP(data, vertices, arcs, F, alpha, K, Q, demands, 
             cluster_result, radius=5, runtime_limit=300, gap_acceptance=None, print_out=True)

Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1948 rows, 1218 columns and 9327 nonzeros
Model fingerprint: 0x5b2e0187
Variable types: 588 continuous, 630 integer (630 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+02, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 1219 rows and 649 columns
Presolve time: 0.04s
Presolved: 729 rows, 569 columns, 5177 nonzeros
Variable types: 261 continuous, 308 integer (308 binary)
Found heuristic solution: objective 2471681.3183
Found heuristic solution: objective 1418089.2502

Root relaxation: objective 7.969118e+05, 911 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

In [ ]:
# with open('.\\ClusteredResult\\som_cluster_result.pickle', 'wb') as handle:
#     pickle.dump(cluster_result, handle)

### Fetch data network and cluster result

In [ ]:
cluster_result_path = ".\\ClusteredResult\\kmed_result.pickle"
data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands, G = fetch_all_dataset()
cluster_result = pickle.load(open(cluster_result_path, "rb"))

In [ ]:
radius = 5

In [ ]:
network, vertices, demands, center_lookup = preprocess_clusters(data, demands, cluster_result, radius)

In [ ]:
arcs = fetch_arcs(network, vertices, center_lookup)

### Run the MILP model

In [ ]:
least_vehicles_needed = math.ceil(max(_demands.values()) / Q[0])
_Q = np.array(Q) * least_vehicles_needed
_F = np.array(F) * least_vehicles_needed
_alpha = np.array(alpha) * least_vehicles_needed

In [ ]:
least_vehicles_needed = math.ceil(max(_demands.values()) / Q[0])
_Q = np.array([np.array(Q) * i for i in range(1, least_vehicles_needed + 1)]).reshape(1, -1)[0]
_F = np.array([np.array(F) * i for i in range(1, least_vehicles_needed + 1)]).reshape(1, -1)[0]
_alpha = np.array([np.array(alpha) * i for i in range(1, least_vehicles_needed + 1)]).reshape(1, -1)[0]
_K = list(range(len(_Q)))

In [ ]:
V_d = [v for v in vertices if v.startswith("D")]
V_c = [v for v in vertices if v not in V_d]

objective_value, solution, runtime, mip_gap = run_F4(demands, vertices, arcs, V_d, V_c, _F, _alpha, _K, _Q,
                                                     runtime_limit=300, print_out=True)
print(objective_value)
print(runtime)
print(mip_gap)
result_dict = {
    "obj_value": objective_value,
    "runtime": runtime,
    "mip_gap": mip_gap,
    "solution": solution
}

In [ ]:
result_dict

In [ ]:
# store the result
json.dump(result_dict, open(".\\Solution\\paris_km_r5_result.json", 'w'))

### Print the map

In [ ]:
file_name = "paris_km_r5_result"

In [ ]:
osm_print_cluster_tours(network, G, cluster_result, result_dict, COLOR_LIST, file_name)

In [ ]:
IFrame(src=f'.\\Solution\\{file_name}.html', width=1000, height=800)